In [1]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np

In [2]:
## Acessando pelo pacote da BD

df_m=bd.read_sql("SELECT ano, tema, sigla_uf,policia_civil_efetivo_mulher FROM `basedosdados-projetos.republica.indicadores_estadic_gerais` where ano=2019 and tema='seguranca_publica' order by sigla_uf", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 27/27 [00:00<00:00, 77.81rows/s]


In [3]:
df_h=bd.read_sql("SELECT ano, tema, sigla_uf,policia_civil_efetivo_homem FROM `basedosdados-projetos.republica.indicadores_estadic_gerais` where ano=2019 and tema='seguranca_publica' order by sigla_uf", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 27/27 [00:00<00:00, 93.08rows/s]


In [5]:
df_m['genero']= 'feminino'
df_m = df_m.rename(columns={'policia_civil_efetivo_mulher':'quantidade_pessoas'})
df_m

,ano,tema,sigla_uf,quantidade_pessoas,genero
0,2019,seguranca_publica,AC,242,feminino
1,2019,seguranca_publica,AL,400,feminino
2,2019,seguranca_publica,AM,878,feminino
3,2019,seguranca_publica,AP,256,feminino
4,2019,seguranca_publica,BA,2383,feminino
5,2019,seguranca_publica,CE,940,feminino
6,2019,seguranca_publica,DF,1266,feminino
7,2019,seguranca_publica,ES,614,feminino
8,2019,seguranca_publica,GO,1522,feminino
9,2019,seguranca_publica,MA,398,feminino


In [6]:
df_h['genero']= 'masculino'
df_h = df_h.rename(columns={'policia_civil_efetivo_homem':'quantidade_pessoas'})
df_h

,ano,tema,sigla_uf,quantidade_pessoas,genero
0,2019,seguranca_publica,AC,531,masculino
1,2019,seguranca_publica,AL,1563,masculino
2,2019,seguranca_publica,AM,1601,masculino
3,2019,seguranca_publica,AP,584,masculino
4,2019,seguranca_publica,BA,4810,masculino
5,2019,seguranca_publica,CE,2724,masculino
6,2019,seguranca_publica,DF,3182,masculino
7,2019,seguranca_publica,ES,1544,masculino
8,2019,seguranca_publica,GO,2936,masculino
9,2019,seguranca_publica,MA,1652,masculino


In [8]:
df = pd.concat([df_h,df_m])

Subindo o arquivo para o gbq

In [9]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 26
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ano                 54 non-null     Int64 
 1   tema                54 non-null     object
 2   sigla_uf            54 non-null     object
 3   quantidade_pessoas  54 non-null     Int64 
 4   genero              54 non-null     object
dtypes: Int64(2), object(3)
memory usage: 2.6+ KB


In [11]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de implementação da legislação.'),
 bigquery.SchemaField('tema','STRING',description='Tema de origem daquele dado na ESTADIC'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
bigquery.SchemaField('quantidade_pessoas','INTEGER',description='Quantitativo total de pessoas'),
 bigquery.SchemaField('genero','STRING',description='Genero daquele quantitativo')

 
 ]

In [12]:
table_ref = dataset_ref.table('ESTADIC_policial_civil_genero')

In [13]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [39]:
#para excluir tabelas
#client.delete_table('perfil_remuneracao.ESTADIC_policial_civil_habitantes', not_found_ok=True)

In [15]:
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=c702423b-9f28-496c-a49f-044682c038c9>